In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import image_dataset_from_directory
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [2]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [3]:
filenames=os.listdir("C:/Users/seroq/Desktop/3._YARI_YIL/AI/kalemkitap/train")

categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='kalem':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 61, 61, 64)       2

In [8]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [11]:
df["category"] = df["category"].replace({0:'kitap',1:'kalem'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=3

In [12]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "C:/Users/seroq/Desktop/3._YARI_YIL/AI/kalemkitap/train",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "C:/Users/seroq/Desktop/3._YARI_YIL/AI/kalemkitap/train", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "C:/Users/seroq/Desktop/3._YARI_YIL/AI/kalemkitap/test",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

Found 48 validated image filenames belonging to 2 classes.
Found 12 validated image filenames belonging to 2 classes.
Found 0 validated image filenames belonging to 0 classes.


C:\Users\seroq\AppData\Roaming\Python\Python39\site-packages\keras\preprocessing\image.py:1137: UserWarning: Found 48 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [13]:
epochs=10
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

C:\Users\seroq\AppData\Local\Temp\ipykernel_13848\2528005629.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
16/16 [==============================] - 6s 310ms/step - loss: 0.9724 - accuracy: 0.7083 - val_loss: 2.3800 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 2/10
16/16 [==============================] - 4s 263ms/step - loss: 0.7050 - accuracy: 0.7917 - val_loss: 5.8106 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 3/10
16/16 [==============================] - 4s 261ms/step - loss: 0.3677 - accuracy: 0.8750 - val_loss: 10.7369 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 4/10
16/16 [==============================] - 4s 271ms/step - loss: 0.2110 - accuracy: 0.9583 - val_loss: 12.4782 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 5/10
16/16 [==============================] - 4s 259ms/step - loss: 0.0379 - accuracy: 0.9792 - val_loss: 13.7669 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 6/10
16/16 [==============================] - 4s 272ms/step - loss: 0.3977 - accuracy: 0.8958 - val_loss: 15.0447 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 7/10
16/16 [==============================] - 4s 261ms/step 

In [15]:
model.save("kalem_kitap_10epoch.h5")